In [5]:
stations_cleaned = pd.read_csv('/Users/bkuo/Local Documents/Bootcamp 2020/Project/Capstone/CitiBikeCapStone/Data/Dock station data/stations_cleaned.csv')
#             delim_whitespace = True)


/Users/bkuo/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
dock = stations_cleaned.copy()
dock.hour = dock.hour.astype(str).apply(lambda x: x.zfill(2))
dock.loc[dock['hour'] == '24', 'hour'] = '00'
dock.minute = dock.minute.astype(str).apply(lambda x: x.zfill(2))
dock['time'] = dock.hour + ':' + dock.minute + ':' +  '00'
dock['datetime'] = pd.to_datetime(dock['date'] + ' ' + dock['time'])
dock = dock.drop(['date', 'hour', 'minute', 'time'], axis = 1)
dock.head()

,dock_id,dock_name,avail_bikes,avail_docks,tot_docks,_lat,_long,in_service,status_key,dock_status,dayofweek,dayofweek2,season,datetime
0,72,W 52 St & 11 Ave,8,31,39,40.767272,-73.993929,1,1,Empty Alert,6,Sunday,spring,2015-03-01 01:06:00
1,72,W 52 St & 11 Ave,8,31,39,40.767272,-73.993929,1,1,Empty Alert,6,Sunday,spring,2015-03-01 01:16:00
2,72,W 52 St & 11 Ave,8,30,38,40.767272,-73.993929,1,1,Empty Alert,6,Sunday,spring,2015-03-01 01:25:00
3,72,W 52 St & 11 Ave,8,30,38,40.767272,-73.993929,1,1,Empty Alert,6,Sunday,spring,2015-03-01 01:35:00
4,72,W 52 St & 11 Ave,8,30,38,40.767272,-73.993929,1,1,Empty Alert,6,Sunday,spring,2015-03-01 01:45:00


In [45]:
# adding features, making target and input data
target = dock.dock_status
target = target.replace(['Empty Alert', 'Healthy', 'Full Alert'], [0, 1, 2])
logit_input = dock[['in_service', 'avail_bikes', 'avail_docks', 'dayofweek', 'season', 'datetime']]
logit_input = pd.get_dummies(logit_input, columns = ['season'], drop_first = True)
logit_input['month'] = logit_input.datetime.dt.month
logit_input['hour'] = logit_input.datetime.dt.hour
logit_input['peak'] = [1 if hr > 5 and hr < 20 else 0 for hr in logit_input['hour']]
logit_input = logit_input.drop('datetime', axis = 1)

In [46]:
# model fitting
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix 

logit = LogisticRegression(multi_class='ovr', solver='liblinear')
params = {'C':np.logspace(-4,4, 20)}
gs_logit = GridSearchCV(logit, params, cv=3, iid=True)

In [47]:
logit_input.head()

,in_service,avail_bikes,avail_docks,dayofweek,season_spring,season_summer,season_winter,month,hour,peak
0,1,8,31,6,1,0,0,3,1,0
1,1,8,31,6,1,0,0,3,1,0
2,1,8,30,6,1,0,0,3,1,0
3,1,8,30,6,1,0,0,3,1,0
4,1,8,30,6,1,0,0,3,1,0


In [48]:
# logistic = LogisticRegression(C=1e4, solver='liblinear', multi_class='auto')
# logistic.fit(logit_input, target)
# logistic.score(logit_input, target)

0.9662842824956646

In [49]:
gs_logit.fit(logit_input, target)

/Users/bkuo/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_search.py:849: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


GridSearchCV(cv=3,
             estimator=LogisticRegression(multi_class='ovr',
                                          solver='liblinear'),
             iid=True,
             param_grid={'C': array([1.00000000e-04, 2.63665090e-04, 6.95192796e-04, 1.83298071e-03,
       4.83293024e-03, 1.27427499e-02, 3.35981829e-02, 8.85866790e-02,
       2.33572147e-01, 6.15848211e-01, 1.62377674e+00, 4.28133240e+00,
       1.12883789e+01, 2.97635144e+01, 7.84759970e+01, 2.06913808e+02,
       5.45559478e+02, 1.43844989e+03, 3.79269019e+03, 1.00000000e+04])})

In [50]:
gs_logit.best_params_

{'C': 0.004832930238571752}

In [51]:
gs_logit.best_score_

0.9652149870177045

In [53]:
gs_logit.best_estimator_.score(logit_input, target)

0.9663049978573031

In [54]:
cm = confusion_matrix(target, gs_logit.best_estimator_.predict(logit_input))
cm

array([[15708898,   187066,      840],
       [  326774,  9973496,   322195],
       [       0,   254554,  5617598]])

In [55]:
## The accuracy against each rating class 0, 1, 2
np.diag(cm/cm.sum(axis=1).reshape((-1,1)))

array([0.98817964, 0.93890599, 0.95665065])

In [56]:
gs_logit.best_estimator_.coef_

array([[ 2.42214383e+00, -2.71607636e+00,  1.16788503e+00,
         5.63011548e-03, -4.94131801e-01, -4.31637343e-01,
        -3.72195760e-01, -4.31772301e-02,  2.92827043e-03,
        -4.26901206e-02],
       [-4.27015068e-01,  9.70112549e-03, -4.61229741e-02,
         3.11261273e-02,  7.79127822e-03, -8.85460468e-03,
         7.54879742e-02,  5.35114675e-03,  1.13800995e-03,
         3.49723396e-02],
       [ 3.83597499e+00,  3.36364915e-01, -7.71052846e-01,
        -3.93787045e-03,  3.78307960e-01, -1.07656600e-01,
         4.80400000e-02, -2.79455200e-02, -1.10454767e-02,
        -1.17059620e-02]])